<a href="https://colab.research.google.com/github/matthew110395/12004210_DataAnalytics/blob/main/12004210_DAOTW_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
This assignment builds on the New York taxi problem identified in assignment 1, where the City of New York is looking for a way to accurately predict the number of collisions on a particular day of the week. Within this document two different types of machine learning models will be utilised to predict the number of collisions. The linear relationships identified in assignment 1 will be used to create a linear regression model. A Deep Learning Neural Network (DNN) will be used to predict the number of collisions where the relationship is not linear. 

#Imports
To prepare data for machine learning the pandas package has been used. The numpy package has been used to aid with mathematical functions.

As within part 1 of this assignment the data file containing location data exceeds the size limit for hosting within github. To overcome this the file was zipped. In order to extract the data from the zip file the zipfile package has been used.

Within this document, Tensorflow has been used for machine learning, with both a linear regression model and a Deep Neural Network model. Tensorflow version 1 is unsupported within Google Colab, therefore must be installed using the pip package manager.

Shutil has also been imported to allow for easy file management, in particular the removal of saved models.

In [87]:
import pandas as pd
import numpy as np
import zipfile


!pip install tensorflow==1.15.2
import tensorflow as tf
# needed for high-level file management
import shutil  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Linear Regressor
Throughout assignment 1 a number of linear relationships were uncovered within the dataset. These relationsips form the basis of the linear regression models below.

A linear regressor is used to predict an output variable based on one or more input variables. REF https://www.ibm.com/uk-en/topics/linear-regression 17/11



To improve the accuracy of the model the target values are scaled or standarised. This reduces the range of collisions from 188-1161 to 0.1619... - 1, this allows for quicker training of the model. https://towardsdatascience.com/understand-data-normalization-in-machine-learning-8ff3062101f0 REF 17.11

In [88]:
SCALE_COLLISIONS=1161

##Precipition
As uncovered in assignment 1, as the volume of precipition increases, the number of collisions increase. 

The cleaned datafile produced in assignment is imported.

In [175]:
df_prcp = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean.csv', index_col=0, )
print(df_prcp[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In order to create the linear regression model, extra columns are removed to simplify the model with the aim of reducing error values within the model.

The incomplete years (2012 and 2022) are removed, along with the eronious data for 2020 and 2021.

To aid with the production of the model the target, in this case the number of collisions is moved to the end of the data table.

The summary below shows there is 2539 rows within the dataset after the data has been cleansed further, as this count exists for each column each column or predcictor contains data.

In [176]:
df_prcp = df_prcp.drop(columns=['collision_date', 'temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp = df_prcp.loc[df_prcp["year"] != 2012]
df_prcp = df_prcp.loc[df_prcp["year"] < 2020]
cols = df_prcp['NUM_COLLISIONS']
df_prcp = df_prcp.drop(columns=['NUM_COLLISIONS'])
df_prcp.insert(loc=9, column='NUM_COLLISIONS', value=cols)
print(df_prcp[:6])
df_prcp.describe()

    day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
49    4  2016   1  28  0.09    0             0                 0     0   
51    5  2014   1  17  0.00    1             0                 0     0   
54    1  2016   1  25  0.02    0             0                 0     0   
55    5  2016   1  29  0.00    0             0                 0     0   
58    5  2017   1  20  0.00    0             0                 0     0   
59    7  2013   1  13  0.01    1             0                 0     0   

    NUM_COLLISIONS  
49             681  
51             589  
54             658  
55             645  
58             605  
59             373  


,day,year,mo,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,3.998425,2015.989366,6.518708,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,599.135093
std,2.003542,1.996126,3.455211,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,100.299164
min,1.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1161.000000


To remove any bias within the dataset, it is randomly shuffled. This shuffled dataset is then split into the predictors and the target.

In [177]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp.iloc[np.random.permutation(len(df_prcp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail
3265    7  2018  11   4  0.45    0             0                 0     0
3614    6  2017  12  16  0.08    0             0                 1     0
429     6  2015   2  21  0.00    0             1                 0     0
1291    4  2014   5   8  0.00    0             1                 0     0
1022    5  2016   4  22  0.00    1             1                 0     0
2192    5  2015   8  14  0.00    0             0                 0     0
      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
3265    7  2018  11   4  0.45    0             0                 0     0   
3614    6  2017  12  16  0.08    0             0                 1     0   
429     6  2015   2  21  0.00    0             1                 0     0   
1291    4  2014   5   8  0.00    0             1                 0     0   
1022    5  2016   4  22  0.00    1             1                 0     0   
2192    5  2015   8  14  0.00    

In [178]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

3265    502
3614    661
429     517
1291    596
1022    741
2192    729
Name: NUM_COLLISIONS, dtype: int64


To test the accuracy/reliabilty of the trained linear regressor, the dataset has been split into a training and a testing dataset. As when compared with other machine learning datasets, this dataset is relatively small a larger portion is required for testing. 20% (511 rows) of the dataset has been reserved for testing, with the remaining 80% (2044 rows) used for training the dataset.

In [179]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 9
# Define the number of output values (targets)
noutputs = 1

2031
508


In [182]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99c4c06d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 0.25712004, step = 1
INFO:tensorflow:global_step/sec: 360.846
INFO:tensorflow:loss = 0.0077984757, step = 101 (0.287 sec)
INFO:tensorflow:global_step/sec: 471.679
INFO:tensorflow:loss = 0.005989602, step = 201 (0.210 sec)
INFO:tensorflow:global_step/sec: 484.663
INFO:tensorflow:loss = 0.0062982165, step = 301 (0.207 sec)
INFO:tensorflow:global_step/sec: 521.284
INFO:tensorflow:loss = 0.007830896, step = 401 (0.190 sec)
INFO:tensorflow:global_step/sec: 694.446
INFO:tensorflow:loss = 0.0070295143, step = 501 (0.141 sec)
INFO:tensorflow:global_step/sec: 811.379
INFO:tensorflow:loss = 0.0074045705, step = 601 (0.125 sec)
INFO:tensorflow:global_step/sec: 795.578
INFO:tensorflow:loss = 0.0066971965, step = 701

LinearRegression has RMSE of 95.58101427895235
Just using average = 599.2466765140325 has RMSE of 102.9435177029394


A number of learning rates between 0.0000001 and 0.1 were been used to determine a suitable learning rate for the dataset. As the learning rate decreases the the overall time to train the dataset increases. REF https://towardsdatascience.com/understanding-learning-rates-and-how-it-improves-performance-in-deep-learning-d0d4059c1c10 17/11

In [181]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99bb8ed90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_prc

508
508
[0.5022183  0.5353434  0.5206734  0.51581275 0.49316192 0.5045507
 0.5338893  0.5600685  0.5156957  0.47868684 0.54077893 0.5094852
 0.5023002  0.5001617  0.51542366 0.53891903 0.5187261  0.50646764
 0.5721638  0.5590161  0.5000497  0.51368797 0.5334992  0.5064387
 0.5656928  0.4869912  0.52234983 0.50545883 0.505442   0.46662718
 0.52208596 0.48154363 0.5144106  0.5650267  0.514192   0.55520886
 0.51249874 0.5646579  0.4953361  0.5488128  0.5561887  0.5030065
 0.5198123  0.51275915 0.4894435  0.4967429  0.5206874  0.56131625
 0.47201726 0.52059764 0.5433232  0.47994936 0.5841606  0.50721246
 0.54845345 0.5150955  0.51543826 0.57824385 0.5476491  0.5040895
 0.51481164 0.55049896 0.52189547 0.49409357 0.57436055 0.53269124
 0.51624864 0.48820978 0.53704005 0.4861982  0.57216585 0.55831945
 0.5313755  0.5380306  0.5825255  0.54509604 0.55864084 0.5512855
 0.59518117 0.4968579  0.5170377  0.53704613 0.5274571  0.5502619
 0.50967604 0.54330957 0.51766753 0.53065425 0.52828157 0.487

Two main tests have been applied to the model. The Route Mean Squared Error (RMSE) and a comparison between the target values in the testing dataset and the predicted values using the predictors in the testing dataset.

Due to the nature of machine learning, different results are achieved with each run of the model.

Predominantly the RMSE of the model is lower than that of the average. This indicates that the linear regression model makes more accurate predictions when compared to using just the average. The error rate of the model is aprocimately -2% when comparing the predicited and actual results.

##Dew Point (dewp)


In [96]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [97]:
df_dewp = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp = df_dewp.loc[df_dewp["year"] != 2012]
df_dewp = df_dewp.loc[df_dewp["year"] < 2020]
cols = df_dewp['NUM_COLLISIONS']
df_dewp = df_dewp.drop(columns=['NUM_COLLISIONS'])
df_dewp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_dewp[:6])
df_dewp.describe()

    day  year  mo  da  dewp  NUM_COLLISIONS
49    4  2016   1  28  24.4             681
51    5  2014   1  17  35.8             589
54    1  2016   1  25  21.2             658
55    5  2016   1  29  36.8             645
58    5  2017   1  20  32.5             605
59    7  2013   1  13  44.9             373


,day,year,mo,da,dewp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.998434,2015.999217,6.524070,15.723679,44.163170,599.109980
std,2.000391,2.000000,3.449676,8.801271,16.995303,100.277185
min,1.000000,2013.000000,1.000000,1.000000,-6.700000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,32.150000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,45.300000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,58.500000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,74.100000,1161.000000


In [98]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_dewp.iloc[np.random.permutation(len(df_dewp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  dewp
3546    2  2013  12   3  38.2
1594    1  2014   6  23  50.3
3652    4  2017  12  14  20.4
2858    1  2014  10   6  42.6
3616    4  2014  12   4  28.0
1997    1  2019   7  29  69.7
      day  year  mo  da  dewp  NUM_COLLISIONS
3546    2  2013  12   3  38.2             553
1594    1  2014   6  23  50.3             644
3652    4  2017  12  14  20.4             883
2858    1  2014  10   6  42.6             639
3616    4  2014  12   4  28.0             606
1997    1  2019   7  29  69.7             649


In [99]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

3546    553
1594    644
3652    883
2858    639
3616    606
1997    649
Name: NUM_COLLISIONS, dtype: int64


In [100]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
511


In [101]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_dewp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99cb169d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 0.2736016, step = 1
INFO:tensorflow:global_step/sec: 480.979
INFO:tensorflow:loss = 0.005087829, step = 101 (0.212 sec)
INFO:tensorflow:global_step/sec: 698.826
INFO:tensorflow:loss = 0.0064100022, step = 201 (0.144 sec)
INFO:tensorflow:global_step/sec: 758.07
INFO:tensorflow:loss = 0.007078087, step = 301 (0.129 sec)
INFO:tensorflow:global_step/sec: 742.854
INFO:tensorflow:loss = 0.00606805, step = 401 (0.136 sec)
INFO:tensorflow:global_step/sec: 889.659
INFO:tensorflow:loss = 0.008043197, step = 501 (0.112 sec)
INFO:tensorflow:global_step/sec: 827.545
INFO:tensorflow:loss = 0.00697574, step = 601 (0.119 sec)
INFO:tensorflow:global_step/sec: 793.297
INFO:tensorflow:loss = 0.0063088676, step = 701 (0.128

LinearRegression has RMSE of 98.15872019343414
Just using average = 598.5503913894324 has RMSE of 101.44774288608059


In [102]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99fd21c90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_dew

511
511
[0.5071501  0.5173477  0.53807575 0.5380053  0.4993347  0.545433
 0.55663407 0.53290254 0.4356096  0.45301634 0.4517532  0.4918821
 0.5560469  0.5714761  0.50050056 0.5597803  0.48728693 0.45440856
 0.4730551  0.49790177 0.5191456  0.48749077 0.5572373  0.52836627
 0.44886997 0.49089465 0.46411264 0.4454686  0.46870852 0.50530314
 0.5014508  0.5300986  0.5470422  0.52702373 0.47059786 0.53770053
 0.50189173 0.4894814  0.44880244 0.5121031  0.4660586  0.5004295
 0.45065013 0.52709985 0.49543202 0.50966686 0.5489253  0.5285865
 0.50540906 0.47518182 0.5196869  0.46776497 0.4537289  0.50808275
 0.45812696 0.48357895 0.513404   0.47737828 0.52357376 0.47163028
 0.46608952 0.5098348  0.44514132 0.4849006  0.4394685  0.48629153
 0.46686152 0.52792996 0.54482913 0.51529807 0.50025    0.5120777
 0.48517406 0.46968082 0.4996432  0.5485673  0.4478021  0.44951943
 0.4677514  0.5065068  0.479767   0.49127606 0.52099854 0.45093644
 0.4841497  0.48392794 0.51630193 0.47214982 0.49802795 0.47

##Sea Level Pressure (SLP)


In [103]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/slp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [104]:
df_slp = df.drop(columns=['collision_date', 'temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_slp = df_slp.loc[df_slp["year"] != 2012]
df_slp = df_slp.loc[df_slp["year"] < 2020]
cols = df_slp['NUM_COLLISIONS']
df_slp = df_slp.drop(columns=['NUM_COLLISIONS'])
df_slp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_slp[:6])
df_slp.describe()

    day  year  mo  da     slp  NUM_COLLISIONS
49    4  2016   1  28  1016.1             681
51    5  2014   1  17  1014.8             589
54    1  2016   1  25  1021.4             658
55    5  2016   1  29   999.4             645
58    5  2017   1  20  1015.5             605
59    7  2013   1  13  1020.7             373


,day,year,mo,da,slp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.999217,2016.000391,6.522114,15.719765,1016.777221,599.147162
std,2.000783,2.000294,3.447986,8.796698,7.628429,100.268048
min,1.000000,2013.000000,1.000000,1.000000,989.500000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,1012.200000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,1016.700000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,1021.700000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,1044.200000,1161.000000


In [105]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_slp.iloc[np.random.permutation(len(df_slp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da     slp
2235    7  2019   8  18  1016.4
1700    4  2013   6  13  1008.2
2292    5  2013   8  16  1021.2
801     6  2014   3  22  1013.6
2278    2  2018   8  14  1013.0
827     3  2017   3  22  1011.4
      day  year  mo  da     slp  NUM_COLLISIONS
2235    7  2019   8  18  1016.4             481
1700    4  2013   6  13  1008.2             534
2292    5  2013   8  16  1021.2             678
801     6  2014   3  22  1013.6             493
2278    2  2018   8  14  1013.0             701
827     3  2017   3  22  1011.4             617


In [106]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2235    481
1700    534
2292    678
801     493
2278    701
827     617
Name: NUM_COLLISIONS, dtype: int64


In [107]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
511


In [108]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_slp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_slp', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99c4b79d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_slp/model.ckpt.
INFO:tensorflow:loss = 0.2676384, step = 1
INFO:tensorflow:global_step/sec: 698.505
INFO:tensorflow:loss = 0.0069545344, step = 101 (0.148 sec)
INFO:tensorflow:global_step/sec: 698.601
INFO:tensorflow:loss = 0.007760483, step = 201 (0.144 sec)
INFO:tensorflow:global_step/sec: 715.28
INFO:tensorflow:loss = 0.009986185, step = 301 (0.137 sec)
INFO:tensorflow:global_step/sec: 857.933
INFO:tensorflow:loss = 0.0059080934, step = 401 (0.119 sec)
INFO:tensorflow:global_step/sec: 808.137
INFO:tensorflow:loss = 0.0066342466, step = 501 (0.122 sec)
INFO:tensorflow:global_step/sec: 569.085
INFO:tensorflow:loss = 0.007184749, step = 601 (0.178 sec)
INFO:tensorflow:global_step/sec: 768.6
INFO:tensorflow:loss = 0.0057589365, step = 701 (0.12

LinearRegression has RMSE of 92.26679987167856
Just using average = 599.5567514677103 has RMSE of 97.76883640443617


In [109]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_slp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99c2923d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_slp/

511
511


INFO:tensorflow:Done running local_init_op.


[0.47868532 0.5333827  0.47306892 0.5544367  0.5448285  0.4938599
 0.48487094 0.5065225  0.59958136 0.5349501  0.5190374  0.5368878
 0.557319   0.4817469  0.5364394  0.49532747 0.54422766 0.5153652
 0.4636602  0.5233125  0.5400646  0.53247744 0.5256451  0.5130015
 0.47018406 0.51014394 0.5302009  0.566365   0.5509676  0.4739107
 0.518703   0.48598227 0.4982097  0.5580803  0.50485474 0.5244103
 0.49123466 0.5386966  0.54834896 0.4541234  0.48790422 0.48690566
 0.52233267 0.5080359  0.51348597 0.5525831  0.5461885  0.5488752
 0.49246898 0.549272   0.56279784 0.48030993 0.54479045 0.55760145
 0.52341986 0.5001317  0.53902364 0.5405909  0.47027388 0.52854306
 0.5455614  0.5151529  0.5095876  0.5034268  0.5468528  0.49195114
 0.49722138 0.49761418 0.54901606 0.4716442  0.5477364  0.506483
 0.48622844 0.51366645 0.478448   0.504023   0.5294683  0.47456506
 0.5344437  0.543488   0.5416919  0.50863916 0.4847067  0.5145234
 0.47311282 0.5585964  0.5223745  0.5609128  0.53938264 0.52208424
 0.51

##Gust

In [110]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/gust_clean.csv', index_col=0, )
print(df[:6])

    day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
3     5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
11    3  2020   1  15     2020-01-15             508  43.9  38.3  1019.4   
12    5  2021   1   1     2021-01-01             257  39.6  29.3  1029.3   
14    2  2022   1  25     2022-01-25             235  41.6  31.8  1013.2   
18    7  2021   1   3     2021-01-03             186  41.1  32.3  1018.0   
19    4  2020   1   2     2020-01-02             413  39.6  28.9  1011.8   

    visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
3    10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
11    8.2  ...  51.1  35.1  0.02  999.9    1             0                 0   
12   10.0  ...  54.0  33.1  0.13  999.9    0             0                 0   
14   10.0  ...  48.9  30.0  0.00  999.9    0             0                 0   
18   10.0  ...  53.1  39.0  0.00  999.9    0             0         

In [111]:
df_gust = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','dewp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_gust = df_gust.loc[df_gust["year"] != 2012]
df_gust = df_gust.loc[df_gust["year"] < 2020]
cols = df_gust['NUM_COLLISIONS']
df_gust = df_gust.drop(columns=['NUM_COLLISIONS'])
df_gust.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_gust[:6])
df_gust.describe()

    day  year  mo  da  gust  NUM_COLLISIONS
74    7  2016   1  17  18.1             451
76    4  2014   1   9  20.0             561
79    6  2019   1  19  21.0             479
80    7  2015   1  11  17.1             341
83    4  2015   1  29  20.0             519
85    7  2019   1  13  15.9             374


,day,year,mo,da,gust,NUM_COLLISIONS
count,1629.000000,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000
mean,4.024555,2015.91283,6.278699,15.702885,27.511602,596.513198
std,1.989070,2.01341,3.747683,8.667634,7.366770,104.479660
min,1.000000,2013.00000,1.000000,1.000000,14.000000,188.000000
25%,2.000000,2014.00000,3.000000,8.000000,22.000000,526.000000
50%,4.000000,2016.00000,6.000000,16.000000,26.000000,597.000000
75%,6.000000,2018.00000,10.000000,23.000000,31.100000,663.000000
max,7.000000,2019.00000,12.000000,31.000000,71.100000,1161.000000


In [112]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_gust.iloc[np.random.permutation(len(df_gust))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  gust
1458    2  2017   5  23  24.1
771     5  2013   3   1  24.1
2105    1  2014   7  28  26.0
461     5  2016   2  19  24.1
2966    1  2014  10  20  27.0
1718    3  2016   6   1  22.0
      day  year  mo  da  gust  NUM_COLLISIONS
1458    2  2017   5  23  24.1             659
771     5  2013   3   1  24.1             603
2105    1  2014   7  28  26.0             595
461     5  2016   2  19  24.1             532
2966    1  2014  10  20  27.0             591
1718    3  2016   6   1  22.0             772


In [113]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1458    659
771     603
2105    595
461     532
2966    591
1718    772
Name: NUM_COLLISIONS, dtype: int64


In [114]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

1303
326


In [115]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_gust', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_gust', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd9af719110>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_gust/model.ckpt.
INFO:tensorflow:loss = 0.27697435, step = 1
INFO:tensorflow:global_step/sec: 430.209
INFO:tensorflow:loss = 0.008280726, step = 101 (0.238 sec)
INFO:tensorflow:global_step/sec: 479.532
INFO:tensorflow:loss = 0.007771803, step = 201 (0.210 sec)
INFO:tensorflow:global_step/sec: 502.789
INFO:tensorflow:loss = 0.007259279, step = 301 (0.202 sec)
INFO:tensorflow:global_step/sec: 451.134
INFO:tensorflow:loss = 0.006926834, step = 401 (0.217 sec)
INFO:tensorflow:global_step/sec: 437.827
INFO:tensorflow:loss = 0.006699074, step = 501 (0.235 sec)
INFO:tensorflow:global_step/sec: 461.495
INFO:tensorflow:loss = 0.0057665287, step = 601 (0.216 sec)
INFO:tensorflow:global_step/sec: 504.612
INFO:tensorflow:loss = 0.006327304, step = 701 (0.

LinearRegression has RMSE of 88.83604642247232
Just using average = 599.2755180353031 has RMSE of 100.03439310079685


In [116]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_gust', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99fe18090>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_gus

326
326


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.4808129  0.49754852 0.5343925  0.45430008 0.45905632 0.5268488
 0.54438007 0.5532203  0.5508805  0.54666233 0.46852517 0.51174146
 0.54744095 0.49077356 0.4979892  0.5045125  0.49858832 0.577427
 0.44949535 0.48807988 0.5096381  0.5187636  0.537857   0.5051454
 0.5123046  0.4882884  0.56278956 0.5625091  0.52163845 0.5567407
 0.44378945 0.46449938 0.4994982  0.5546624  0.46106714 0.4986182
 0.46910784 0.50934947 0.55493456 0.54278696 0.494294   0.49253547
 0.4881124  0.5327354  0.54614097 0.5154723  0.50101584 0.47131452
 0.52982837 0.48684943 0.5194375  0.45543742 0.4643467  0.53313166
 0.5352727  0.47446644 0.53988695 0.49811885 0.5139935  0.46114302
 0.5776381  0.4690111  0.553347   0.46265244 0.4743408  0.4991158
 0.53799284 0.5412227  0.5309977  0.5665094  0.5566231  0.5114611
 0.55724216 0.5116754  0.4983055  0.50266916 0.4490461  0.54756314
 0.49480188 0.4479491  0.454196   0.49158782 0.5499064  0.46162555
 0.55901235 0.4564303  0.54813904 0.4593838  0.5548011  0.46421427
 0.

##Maximum Sustained Wind Speed (Mxpsd)

In [117]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/mxpsd_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [118]:
df_mxpsd = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','gust','dewp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_mxpsd = df_mxpsd.loc[df_mxpsd["year"] != 2012]
df_mxpsd = df_mxpsd.loc[df_mxpsd["year"] < 2020]
cols = df_mxpsd['NUM_COLLISIONS']
df_mxpsd = df_mxpsd.drop(columns=['NUM_COLLISIONS'])
df_mxpsd.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_mxpsd[:6])
df_mxpsd.describe()

    day  year  mo  da  mxpsd  NUM_COLLISIONS
49    4  2016   1  28    8.9             681
51    5  2014   1  17    8.9             589
54    1  2016   1  25    8.9             658
55    5  2016   1  29    9.9             645
58    5  2017   1  20    9.9             605
59    7  2013   1  13    9.9             373


,day,year,mo,da,mxpsd,NUM_COLLISIONS
count,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000
mean,3.999608,2016.001567,6.520564,15.737172,17.240110,599.033686
std,2.001469,2.000587,3.449204,8.797367,5.858333,100.284761
min,1.000000,2013.000000,1.000000,1.000000,5.100000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,13.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,15.900000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,20.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,49.000000,1161.000000


In [119]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_mxpsd.iloc[np.random.permutation(len(df_mxpsd))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  mxpsd
1631    4  2013   6  20   12.0
2824    1  2015  10  12   11.1
2274    7  2018   8  26   11.1
1077    7  2015   4   5   18.1
1778    3  2015   6   3   19.0
578     1  2019   2  25   31.1
      day  year  mo  da  mxpsd  NUM_COLLISIONS
1631    4  2013   6  20   12.0             681
2824    1  2015  10  12   11.1             505
2274    7  2018   8  26   11.1             528
1077    7  2015   4   5   18.1             422
1778    3  2015   6   3   19.0             658
578     1  2019   2  25   31.1             700


In [120]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1631    681
2824    505
2274    528
1077    422
1778    658
578     700
Name: NUM_COLLISIONS, dtype: int64


In [121]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2042
511


In [122]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_mxpsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_mxpsd', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99fe2fdd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_mxpsd/model.ckpt.
INFO:tensorflow:loss = 0.26019302, step = 1
INFO:tensorflow:global_step/sec: 685.071
INFO:tensorflow:loss = 0.00679522, step = 101 (0.149 sec)
INFO:tensorflow:global_step/sec: 763.159
INFO:tensorflow:loss = 0.0066580754, step = 201 (0.132 sec)
INFO:tensorflow:global_step/sec: 581.708
INFO:tensorflow:loss = 0.0076028043, step = 301 (0.172 sec)
INFO:tensorflow:global_step/sec: 767.354
INFO:tensorflow:loss = 0.007029012, step = 401 (0.132 sec)
INFO:tensorflow:global_step/sec: 692.29
INFO:tensorflow:loss = 0.0071098125, step = 501 (0.143 sec)
INFO:tensorflow:global_step/sec: 761.215
INFO:tensorflow:loss = 0.0067186514, step = 601 (0.132 sec)
INFO:tensorflow:global_step/sec: 738.431
INFO:tensorflow:loss = 0.0060497043, step = 701 

LinearRegression has RMSE of 99.54707597846121
Just using average = 599.8344760039178 has RMSE of 106.6443345361231


In [123]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_mxpsd', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99fce96d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_mx

511
511


INFO:tensorflow:Done running local_init_op.


[0.48685554 0.5017715  0.5195437  0.53615284 0.5454602  0.4770372
 0.5478919  0.51928663 0.4630683  0.5554678  0.53757876 0.51046175
 0.4606119  0.53223115 0.5676637  0.49264532 0.5143805  0.55177903
 0.49720487 0.5292196  0.5016322  0.4959558  0.5637739  0.5187303
 0.4729221  0.5310823  0.5245323  0.54790014 0.4961628  0.5474171
 0.5230903  0.517643   0.50107825 0.47332734 0.4714707  0.52177274
 0.5512216  0.5066799  0.5274798  0.46867326 0.53792155 0.5261571
 0.5013206  0.51682    0.46757972 0.5208199  0.5042332  0.51943666
 0.5045486  0.5179419  0.53066957 0.5310577  0.49924272 0.4973693
 0.47977316 0.5068482  0.49414656 0.5599052  0.5156741  0.48933315
 0.52164257 0.525553   0.47889602 0.5219304  0.5284055  0.54281825
 0.46601078 0.45353243 0.53122807 0.5408706  0.4562273  0.51327634
 0.50791854 0.51089054 0.51573545 0.5449649  0.53421545 0.550504
 0.50065804 0.5429421  0.47455156 0.5280237  0.4997188  0.5393757
 0.4974781  0.5459225  0.53021854 0.5596765  0.47312063 0.5672009
 0.4

##Others


#Deep Learning Neural Network (DNN)

####Precipition (prcp)

In [124]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [125]:
df_prcp_dnn = df.drop(columns=['temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] != 2012]
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] < 2020]
cols = df_prcp_dnn['NUM_COLLISIONS']
df_prcp_dnn = df_prcp_dnn.drop(columns=['NUM_COLLISIONS'])
df_prcp_dnn.insert(loc=25, column='NUM_COLLISIONS', value=cols)
print(df_prcp_dnn[:6])
df_prcp_dnn.describe()

    year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  Dec  \
49  2016  28  0.09    0             0                 0     0    0    0    0   
51  2014  17  0.00    1             0                 0     0    0    0    0   
54  2016  25  0.02    0             0                 0     0    0    0    0   
55  2016  29  0.00    0             0                 0     0    0    0    0   
58  2017  20  0.00    0             0                 0     0    0    0    0   
59  2013  13  0.01    1             0                 0     0    0    0    0   

    ...  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49  ...    0    0    0    0    0    0    0    0    1             681  
51  ...    0    0    0    0    0    0    1    0    0             589  
54  ...    0    0    0    0    0    1    0    0    0             658  
55  ...    0    0    0    0    0    0    1    0    0             645  
58  ...    0    0    0    0    0    0    1    0    0             605  
59  ...    0 

,year,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,Apr,Aug,Dec,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,...,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,2015.989366,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,0.082316,0.083497,0.085467,...,0.082710,0.085467,0.079953,0.142970,0.143364,0.143757,0.142182,0.142576,0.142182,599.135093
std,1.996126,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,0.274899,0.276687,0.279630,...,0.275497,0.279630,0.271273,0.350111,0.350512,0.350913,0.349305,0.349709,0.349305,100.299164
min,2013.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [126]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp_dnn.iloc[np.random.permutation(len(df_prcp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  \
2529  2017  27  0.00    1             0                 0     0    0    0   
3057  2014   8  0.07    1             1                 0     0    0    0   
751   2019   6  0.00    0             0                 0     0    0    0   
2919  2016  30  0.00    0             1                 0     0    0    0   
3454  2013   1  0.00    0             1                 0     0    0    0   
993   2019  17  0.00    0             0                 0     0    1    0   

      Dec  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
2529    0  ...    0    0    0    1    0    0    0    0    1    0  
3057    0  ...    0    0    1    0    0    0    0    0    1    0  
751     0  ...    0    0    0    0    0    0    0    0    1    0  
2919    0  ...    0    0    1    0    0    1    0    0    0    0  
3454    1  ...    0    0    0    0    0    1    0    0    0    0  
993     0  ...    0    0    0    0    0    0    0    0    

In [127]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2529    696
3057    639
751     589
2919    557
3454    420
993     621
Name: NUM_COLLISIONS, dtype: int64


In [128]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

25


In [129]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99cba52d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 5459.094, step = 1
INFO:tensorflow:global_step/sec: 402.258
INFO:tensorflow:loss = 0.01564254, step = 101 (0.256 sec)
INFO:tensorflow:global_step/sec: 490.193
INFO:tensorflow:loss = 0.012573941, step = 201 (0.203 sec)
INFO:tensorflow:global_step/sec: 526.106
INFO:tensorflow:loss = 0.014447069, step = 301 (0.190 sec)
INFO:tensorflow:global_step/sec: 533.257
INFO:tensorflow:loss = 0.010319462, step = 401 (0.187 sec)
INFO:tensorflow:global_step/sec: 555.588
INFO:tensorflow:loss = 0.012983334, step = 501 (0.179 sec)
INFO:tensorflow:global_step/sec: 472.578
INFO:tensorflow:loss = 0.014266018, step = 601 (0.211 sec)
INFO:tensorflow:global_step/sec: 427.622
INFO:tensorflow:loss = 0.009516504, step = 701 (0.233 sec

DNNRegression has RMSE of 83.19526445276746
Just using average = 599.2383062530773 has RMSE of 101.84652506854763


In [130]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99c02a650>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


508
508


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_prcp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5185849  0.5309962  0.5191433  0.51249814 0.54601103 0.55444384
 0.540207   0.50155026 0.49606127 0.5438419  0.5246221  0.5095525
 0.5004417  0.5425191  0.5442233  0.4401011  0.5079435  0.5368232
 0.53625894 0.5224062  0.51932055 0.5204955  0.5187052  0.525364
 0.5145357  0.5248027  0.49972147 0.44550538 0.4923159  0.5230683
 0.53095466 0.43399203 0.55286604 0.51337296 0.49383473 0.5277997
 0.43590927 0.5147517  0.49224252 0.5415322  0.53711516 0.46420795
 0.5021202  0.52774054 0.5479791  0.47053337 0.5150261  0.5766055
 0.5244117  0.5454013  0.52833426 0.5122015  0.5113655  0.5259783
 0.55864173 0.5432283  0.5341896  0.482867   0.49213266 0.43619227
 0.43655252 0.48355663 0.4972222  0.51755255 0.48695058 0.52421856
 0.511749   0.5399346  0.45272064 0.51803327 0.5284241  0.52026945
 0.54353994 0.5543295  0.5280793  0.5172137  0.49829543 0.5373758
 0.53211796 0.5152362  0.5297122  0.51960576 0.49036413 0.43316674
 0.5031002  0.5366728  0.52114356 0.50552875 0.5348181  0.47720975
 0.5

##Dew Point (dewp)


In [131]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [132]:
df_dewp_dnn = df.drop(columns=['temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] != 2012]
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] < 2020]
cols = df_dewp_dnn['NUM_COLLISIONS']
df_dewp_dnn = df_dewp_dnn.drop(columns=['NUM_COLLISIONS'])
df_dewp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_dewp_dnn[:6])
df_dewp_dnn.describe()

    year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  24.4    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  35.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  21.2    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29  36.8    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  32.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  44.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,dewp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2015.999217,15.723679,44.163170,0.082192,0.084932,0.084932,0.077104,0.084932,0.084540,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142857,599.109980
std,2.000000,8.801271,16.995303,0.274710,0.278834,0.278834,0.266808,0.278834,0.278251,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349996,100.277185
min,2013.000000,1.000000,-6.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,32.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,45.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,58.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,74.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [133]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_dewp_dnn.iloc[np.random.permutation(len(df_dewp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
3517  2017  17  13.8    0    0    1    0    0    0    0  ...    0    0    0   
1448  2018  27  53.4    0    0    0    0    0    0    0  ...    1    0    0   
2380  2016   6  67.5    0    1    0    0    0    0    0  ...    0    0    0   
2338  2015  30  66.8    0    1    0    0    0    0    0  ...    0    0    0   
3273  2019   9  21.8    0    0    0    0    0    0    0  ...    0    1    0   
190   2018  16  28.9    0    0    0    0    1    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
3517    0    0    1    0    0    0    0  
1448    0    0    1    0    0    0    0  
2380    0    0    0    0    0    0    0  
2338    0    0    1    0    0    0    0  
3273    0    0    0    0    0    0    0  
190     0    1    0    0    0    0    0  

[6 rows x 21 columns]


In [134]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

3517    547
1448    487
2380    578
2338    563
3273    502
190     617
Name: NUM_COLLISIONS, dtype: int64


In [135]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [136]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_dewp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99fce9090>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 29.350513, step = 1
INFO:tensorflow:global_step/sec: 344.184
INFO:tensorflow:loss = 0.18778694, step = 101 (0.294 sec)
INFO:tensorflow:global_step/sec: 506.029
INFO:tensorflow:loss = 0.1630961, step = 201 (0.199 sec)
INFO:tensorflow:global_step/sec: 463.569
INFO:tensorflow:loss = 0.116834104, step = 301 (0.216 sec)
INFO:tensorflow:global_step/sec: 477.675
INFO:tensorflow:loss = 0.093510956, step = 401 (0.207 sec)
INFO:tensorflow:global_step/sec: 566.45
INFO:tensorflow:loss = 0.0945691, step = 501 (0.179 sec)
INFO:tensorflow:global_step/sec: 570.967
INFO:tensorflow:loss = 0.053805728, step = 601 (0.175 sec)
INFO:tensorflow:global_step/sec: 588.036
INFO:tensorflow:loss = 0.046030343, step = 701 (0.171 sec)
IN

DNNRegression has RMSE of 185.74080399524615
Just using average = 598.4916829745597 has RMSE of 101.7999841627072


In [137]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99c625750>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_dewp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.40849167 0.44215918 0.37407708 0.4081313  0.3998119  0.3918956
 0.3726808  0.36164984 0.35037935 0.3862851  0.4267058  0.39134428
 0.30307537 0.3951958  0.38250843 0.38288587 0.38121256 0.42358023
 0.39098942 0.32479596 0.32256114 0.38163537 0.31474298 0.4147664
 0.41180912 0.27140895 0.43794987 0.34250298 0.30274653 0.38929245
 0.30164096 0.3575675  0.3681231  0.34205338 0.33789164 0.40898722
 0.29605636 0.28531247 0.32907474 0.3401113  0.3421184  0.37710273
 0.39944243 0.3238937  0.38000247 0.40640464 0.31919488 0.39959532
 0.3822829  0.42865962 0.40585518 0.41161317 0.41226757 0.36980802
 0.3503192  0.32974908 0.39536592 0.40672153 0.3765538  0.37783968
 0.35563898 0.396242   0.4327144  0.38217294 0.429427   0.43680263
 0.4520945  0.37521145 0.42123803 0.39553756 0.34904563 0.3435706
 0.27222505 0.4069332  0.29447147 0.32504317 0.34765205 0.35422838
 0.39336738 0.30587947 0.28249878 0.28717044 0.4049695  0.31244603
 0.32807747 0.4132818  0.46986586 0.40407854 0.39883104 0.3911832

##Sea Level Pressure(slp)


In [138]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/slp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [139]:
df_slp_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_slp_dnn = df_slp_dnn.loc[df_slp_dnn["year"] != 2012]
df_slp_dnn = df_slp_dnn.loc[df_slp_dnn["year"] < 2020]
cols = df_slp_dnn['NUM_COLLISIONS']
df_slp_dnn = df_slp_dnn.drop(columns=['NUM_COLLISIONS'])
df_slp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_slp_dnn[:6])
df_slp_dnn.describe()

    year  da     slp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  1016.1    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  1014.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  1021.4    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29   999.4    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  1015.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  1020.7    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,slp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2016.000391,15.719765,1016.777221,0.082192,0.084932,0.084540,0.077104,0.084932,0.084932,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142466,599.147162
std,2.000294,8.796698,7.628429,0.274710,0.278834,0.278251,0.266808,0.278834,0.278834,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349596,100.268048
min,2013.000000,1.000000,989.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,1012.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,1016.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,1021.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,1044.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [140]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_slp_dnn.iloc[np.random.permutation(len(df_slp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da     slp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
2238  2018   6  1018.6    0    1    0    0    0    0    0  ...    0    0    0   
1874  2013  31  1019.3    0    0    0    0    0    1    0  ...    0    0    0   
3166  2014  15  1021.7    0    0    0    0    0    0    0  ...    0    1    0   
2044  2016  25  1014.0    0    0    0    0    0    1    0  ...    0    0    0   
1482  2014  13  1022.0    0    0    0    0    0    0    0  ...    1    0    0   
468   2015  23  1020.2    0    0    0    1    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
2238    0    0    0    1    0    0    0  
1874    0    0    0    0    0    1    0  
3166    0    0    0    0    0    0    0  
2044    0    0    0    1    0    0    0  
1482    0    1    0    0    0    0    0  
468     0    0    0    1    0    0    0  

[6 rows x 21 columns]


In [141]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2238    632
1874    553
3166    529
2044    698
1482    560
468     574
Name: NUM_COLLISIONS, dtype: int64


In [142]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [143]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_slp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_slp', hidden_units=[19,15,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd9af719890>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_slp/model.ckpt.
INFO:tensorflow:loss = 3157.8977, step = 1
INFO:tensorflow:global_step/sec: 286.662
INFO:tensorflow:loss = 0.81244874, step = 101 (0.351 sec)
INFO:tensorflow:global_step/sec: 430.593
INFO:tensorflow:loss = 0.55703247, step = 201 (0.234 sec)
INFO:tensorflow:global_step/sec: 245.196
INFO:tensorflow:loss = 0.3602442, step = 301 (0.405 sec)
INFO:tensorflow:global_step/sec: 465.798
INFO:tensorflow:loss = 0.21667351, step = 401 (0.216 sec)
INFO:tensorflow:global_step/sec: 447.114
INFO:tensorflow:loss = 0.09068417, step = 501 (0.225 sec)
INFO:tensorflow:global_step/sec: 492.975
INFO:tensorflow:loss = 0.047328938, step = 601 (0.199 sec)
INFO:tensorflow:global_step/sec: 419.662
INFO:tensorflow:loss = 0.02520203, step = 701 (0.239 sec)
INFO

DNNRegression has RMSE of 77.96618777117176
Just using average = 598.7592954990215 has RMSE of 94.24576970337212


In [144]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_slp', hidden_units=[19,15,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd9a36ff850>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_slp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.50069565 0.52468026 0.4782404  0.42154166 0.5360888  0.5616261
 0.53668207 0.46657005 0.52105576 0.5324824  0.5085892  0.53905237
 0.48806807 0.49131766 0.5450348  0.49998894 0.5401188  0.38731918
 0.53999186 0.42192474 0.44779438 0.5156937  0.41259292 0.5670615
 0.53810984 0.39906856 0.5110862  0.4731031  0.49670663 0.48035875
 0.5104276  0.48828715 0.49793246 0.5445526  0.46972886 0.5415081
 0.5368218  0.4681412  0.53912127 0.42641705 0.52203685 0.5365205
 0.48546913 0.5189426  0.5420701  0.507533   0.52597064 0.52741885
 0.5638306  0.5217331  0.4110508  0.5317384  0.53559625 0.5131709
 0.5296     0.53853154 0.5483497  0.51151603 0.49285224 0.51545334
 0.5153801  0.51092094 0.52225244 0.5275827  0.54744244 0.5221239
 0.551523   0.572836   0.5409065  0.545136   0.52116823 0.49935326
 0.4111073  0.53205353 0.416811   0.511338   0.5136957  0.53579295
 0.46566176 0.5402014  0.516182   0.5592507  0.5937876  0.46886402
 0.57495433 0.41902766 0.4894625  0.4948925  0.524353   0.43682188
 

##Gust

In [145]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/gust_clean_dnn.csv', index_col=0, )
print(df[:6])

    year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd  gust  \
3   2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0  20.0   
11  2020  15             508  43.9  38.3  1019.4    8.2   5.4   14.0  15.0   
12  2021   1             257  39.6  29.3  1029.3   10.0   7.6   14.0  20.0   
14  2022  25             235  41.6  31.8  1013.2   10.0   9.6   15.0  19.0   
18  2021   3             186  41.1  32.3  1018.0   10.0  10.3   19.0  27.0   
19  2020   2             413  39.6  28.9  1011.8   10.0  13.0   19.0  26.0   

    ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
3   ...    0    0    0    0    0    0    0    1    0    0  
11  ...    0    0    0    0    0    0    0    0    1    0  
12  ...    0    0    0    0    0    0    0    1    0    0  
14  ...    0    0    0    0    1    0    0    0    0    0  
18  ...    0    0    0    0    0    1    0    0    0    0  
19  ...    0    0    0    0    0    0    0    0    0    1  

[6 rows x 38 columns]


In [146]:
df_gust_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_gust_dnn = df_gust_dnn.loc[df_gust_dnn["year"] != 2012]
df_gust_dnn = df_gust_dnn.loc[df_gust_dnn["year"] < 2020]
cols = df_gust_dnn['NUM_COLLISIONS']
df_gust_dnn = df_gust_dnn.drop(columns=['NUM_COLLISIONS'])
df_gust_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_gust_dnn[:6])
df_gust_dnn.describe()

    year  da  gust  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
74  2016  17  18.1    0    0    0    0    1    0    0  ...    0    0    0   
76  2014   9  20.0    0    0    0    0    1    0    0  ...    0    0    0   
79  2019  19  21.0    0    0    0    0    1    0    0  ...    0    0    0   
80  2015  11  17.1    0    0    0    0    1    0    0  ...    0    0    0   
83  2015  29  20.0    0    0    0    0    1    0    0  ...    0    0    0   
85  2019  13  15.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
74    0    1    0    0    0    0             451  
76    0    0    0    0    0    1             561  
79    0    0    0    0    0    0             479  
80    0    1    0    0    0    0             341  
83    0    0    0    0    0    1             519  
85    0    1    0    0    0    0             374  

[6 rows x 22 columns]


,year,da,gust,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,...,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000
mean,2015.91283,15.702885,27.511602,0.095764,0.042357,0.104359,0.095150,0.108656,0.046041,0.061387,...,0.096378,0.087784,0.071209,0.139963,0.141191,0.139963,0.151627,0.138122,0.145488,596.513198
std,2.01341,8.667634,7.366770,0.294358,0.201465,0.305819,0.293513,0.311302,0.209637,0.240113,...,0.295200,0.283067,0.257253,0.347055,0.348325,0.347055,0.358769,0.345133,0.352700,104.479660
min,2013.00000,1.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.00000,8.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,526.000000
50%,2016.00000,16.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,597.000000
75%,2018.00000,23.000000,31.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,663.000000
max,2019.00000,31.000000,71.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [147]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_gust_dnn.iloc[np.random.permutation(len(df_gust_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  gust  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
3316  2013  28  35.9    0    0    0    0    0    0    0  ...    0    1    0   
2667  2015  12  22.9    0    0    0    0    0    0    0  ...    0    0    0   
1369  2015  11  21.0    0    0    0    0    0    0    0  ...    1    0    0   
3297  2019  24  35.9    0    0    0    0    0    0    0  ...    0    1    0   
763   2013  13  26.0    0    0    0    0    0    0    0  ...    0    0    0   
3599  2017  23  28.9    0    0    1    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
3316    0    0    0    0    0    0    1  
2667    1    0    0    0    0    0    0  
1369    0    0    0    1    0    0    0  
3297    0    0    1    0    0    0    0  
763     0    0    0    0    0    1    0  
3599    0    0    0    0    0    0    0  

[6 rows x 21 columns]


In [148]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

3316    429
2667    554
1369    645
3297    517
763     636
3599    565
Name: NUM_COLLISIONS, dtype: int64


In [149]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [150]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_gust', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99c8de790>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_gust/model.ckpt.
INFO:tensorflow:loss = 16477.111, step = 1
INFO:tensorflow:global_step/sec: 389.548
INFO:tensorflow:loss = 0.027402423, step = 101 (0.263 sec)
INFO:tensorflow:global_step/sec: 518.164
INFO:tensorflow:loss = 0.01917019, step = 201 (0.192 sec)
INFO:tensorflow:global_step/sec: 508.984
INFO:tensorflow:loss = 0.021825876, step = 301 (0.198 sec)
INFO:tensorflow:global_step/sec: 536.633
INFO:tensorflow:loss = 0.031008393, step = 401 (0.184 sec)
INFO:tensorflow:global_step/sec: 596.194
INFO:tensorflow:loss = 0.027500287, step = 501 (0.170 sec)
INFO:tensorflow:global_step/sec: 531.292
INFO:tensorflow:loss = 0.021552578, step = 601 (0.188 sec)
INFO:tensorflow:global_step/sec: 573.379
INFO:tensorflow:loss = 0.017725224, step = 701 (0.171 se

DNNRegression has RMSE of 84.87980838918082
Just using average = 594.6139677666922 has RMSE of 97.2516195733968


In [151]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99fe04ad0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


326
326


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_gust/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5295681  0.50279295 0.5186813  0.5550675  0.50926286 0.51478094
 0.34972587 0.5107387  0.48962733 0.48202622 0.5077578  0.45241514
 0.5303733  0.5249841  0.4819516  0.42660913 0.3957548  0.50908947
 0.5008848  0.5027327  0.47737455 0.5260839  0.37693933 0.42754543
 0.42254457 0.5017034  0.48117942 0.48468918 0.5377923  0.40605268
 0.49176955 0.5660472  0.5093014  0.51203895 0.5470723  0.53878564
 0.57368875 0.35110134 0.5360532  0.5356462  0.5778374  0.5130379
 0.5725712  0.5003758  0.47311938 0.5116311  0.47152215 0.5449174
 0.38731432 0.5207215  0.5822002  0.45452356 0.48952582 0.50627136
 0.49711028 0.42456427 0.5276534  0.4993987  0.5580308  0.5286135
 0.530842   0.51208967 0.4694868  0.5561785  0.51324785 0.3787415
 0.50695443 0.51553756 0.426965   0.5357587  0.5119739  0.47602448
 0.5302774  0.5139575  0.50684    0.4626025  0.5006992  0.50202507
 0.45507282 0.5445356  0.5024633  0.46035033 0.48702568 0.54096586
 0.49137062 0.47078678 0.46906513 0.53660595 0.49413583 0.56208515

##Maximum Sustained Wind Speed (mxpsd)

In [152]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/mxpsd_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [153]:
df_mxpsd_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_mxpsd_dnn = df_mxpsd_dnn.loc[df_mxpsd_dnn["year"] != 2012]
df_mxpsd_dnn = df_mxpsd_dnn.loc[df_mxpsd_dnn["year"] < 2020]
cols = df_mxpsd_dnn['NUM_COLLISIONS']
df_mxpsd_dnn = df_mxpsd_dnn.drop(columns=['NUM_COLLISIONS'])
df_mxpsd_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_mxpsd_dnn[:6])
df_mxpsd_dnn.describe()

    year  da  mxpsd  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28    8.9    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17    8.9    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25    8.9    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29    9.9    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20    9.9    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13    9.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,mxpsd,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,...,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000
mean,2016.001567,15.737172,17.240110,0.082256,0.084998,0.084998,0.077164,0.084998,0.084998,0.082256,...,0.081864,0.084998,0.081473,0.143361,0.142969,0.142969,0.142969,0.142577,0.142186,599.033686
std,2.000587,8.797367,5.858333,0.274808,0.278933,0.278933,0.266904,0.278933,0.278933,0.274808,...,0.274212,0.278933,0.273613,0.350509,0.350110,0.350110,0.350110,0.349710,0.349309,100.284761
min,2013.000000,1.000000,5.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,15.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,49.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [154]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_mxpsd_dnn.iloc[np.random.permutation(len(df_mxpsd_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  mxpsd  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
2673  2017  29   17.1    0    0    0    0    0    0    0  ...    0    0    0   
3007  2014   9   21.0    0    0    0    0    0    0    0  ...    0    0    1   
58    2017  20    9.9    0    0    0    0    1    0    0  ...    0    0    0   
2529  2017  27    9.9    0    0    0    0    0    0    0  ...    0    0    0   
3533  2017  24   18.1    0    0    1    0    0    0    0  ...    0    0    0   
870   2018  14   31.1    0    0    0    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
2673    1    0    0    0    1    0    0  
3007    0    0    0    0    0    0    1  
58      0    0    0    0    1    0    0  
2529    1    0    0    0    0    1    0  
3533    0    0    1    0    0    0    0  
870     0    0    0    0    0    1    0  

[6 rows x 21 columns]


In [155]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2673    757
3007    499
58      605
2529    696
3533    469
870     652
Name: NUM_COLLISIONS, dtype: int64


In [156]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [157]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_mxpsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_mxpsd', hidden_units=[19,15,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99fe18510>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_mxpsd/model.ckpt.
INFO:tensorflow:loss = 387.63486, step = 1
INFO:tensorflow:global_step/sec: 134.491
INFO:tensorflow:loss = 0.2524597, step = 101 (0.761 sec)
INFO:tensorflow:global_step/sec: 128.19
INFO:tensorflow:loss = 0.2283586, step = 201 (0.768 sec)
INFO:tensorflow:global_step/sec: 191.647
INFO:tensorflow:loss = 0.21215409, step = 301 (0.526 sec)
INFO:tensorflow:global_step/sec: 217.601
INFO:tensorflow:loss = 0.17387179, step = 401 (0.459 sec)
INFO:tensorflow:global_step/sec: 222.618
INFO:tensorflow:loss = 0.16214314, step = 501 (0.454 sec)
INFO:tensorflow:global_step/sec: 185.284
INFO:tensorflow:loss = 0.13822445, step = 601 (0.537 sec)
INFO:tensorflow:global_step/sec: 149.462
INFO:tensorflow:loss = 0.11480613, step = 701 (0.683 sec)
INFO:

DNNRegression has RMSE of 99.44888454495245
Just using average = 598.5901077375122 has RMSE of 99.44707720459567


In [158]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99caa5d90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_gust/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.42953092 0.58093435 0.547737   0.42803988 0.50829405 0.55152094
 0.6126497  0.43592477 0.5476345  0.53471494 0.56686586 0.47182697
 0.39176238 0.4693615  0.3729343  0.54688    0.42072406 0.45294148
 0.48109403 0.47650725 0.5542305  0.46227285 0.49462044 0.42985213
 0.5012755  0.5346714  0.48129952 0.56147915 0.35417247 0.56215984
 0.5175171  0.5142127  0.49538976 0.5126357  0.5002532  0.55393535
 0.5769882  0.47956684 0.3767095  0.46287817 0.4863152  0.55253226
 0.5028946  0.3930714  0.5127156  0.5674427  0.4293223  0.549355
 0.5250859  0.4948222  0.5009982  0.4990203  0.54888207 0.50419754
 0.5367106  0.53662014 0.5177856  0.5170085  0.5025372  0.49461117
 0.5165586  0.47604793 0.48499322 0.49749807 0.5238497  0.5394536
 0.48843837 0.5654981  0.47369888 0.5106676  0.5085487  0.5107725
 0.6098803  0.61360294 0.4815458  0.38714176 0.54141885 0.4820989
 0.48027164 0.4528684  0.5759838  0.39095393 0.52050817 0.5397778
 0.5009769  0.47250527 0.50374    0.5533272  0.50827444 0.51172245
 

##Others

In [159]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/datadnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [160]:
dnn = df.drop(columns=[ 'prcp', 'dewp','mxpsd','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
dnn = dnn.loc[dnn["year"] != 2012]
dnn = dnn.loc[dnn["year"] < 2020]
cols = dnn['NUM_COLLISIONS']
dnn = dnn.drop(columns=['NUM_COLLISIONS'])
dnn.insert(loc=26, column='NUM_COLLISIONS', value=cols)
print(dnn[:6])
dnn.describe()

    year  da  temp  visib  wdsp   max   min   sndp  Apr  Aug  ...  Nov  Oct  \
49  2016  28  35.0   10.0   4.3  46.0  23.0  999.9    0    0  ...    0    0   
51  2014  17  38.6    6.7   3.7  44.1  32.0  999.9    0    0  ...    0    0   
54  2016  25  33.5   10.0   6.5  37.9  30.0  999.9    0    0  ...    0    0   
55  2016  29  41.3   10.0   5.9  45.0  23.0  999.9    0    0  ...    0    0   
58  2017  20  39.9   10.0   4.3  45.0  37.0  999.9    0    0  ...    0    0   
59  2013  13  45.4    4.3   5.8  46.9  44.1  999.9    0    0  ...    0    0   

    Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    0    1             681  
51    0    0    0    0    1    0    0             589  
54    0    0    0    1    0    0    0             658  
55    0    0    0    0    1    0    0             645  
58    0    0    0    0    1    0    0             605  
59    0    0    1    0    0    0    0             373  

[6 rows x 27 columns]


,year,da,temp,visib,wdsp,max,min,sndp,Apr,Aug,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2556.0,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2.556000e+03,2556.000000,2556.000000,...,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000
mean,2016.0,15.725743,51.487715,8.295618,10.682864,59.564280,43.869757,9.999000e+02,0.082160,0.084898,...,0.082160,0.084898,0.082160,0.143192,0.142801,0.142801,0.142801,0.142801,0.142801,599.118936
std,2.0,8.800168,14.162738,2.207870,4.242687,14.279867,14.722751,2.274182e-13,0.274661,0.278785,...,0.274661,0.278785,0.274661,0.350338,0.349939,0.349939,0.349939,0.349939,0.349939,100.258581
min,2013.0,1.000000,5.800000,0.200000,2.700000,18.000000,-2.000000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.0,8.000000,40.400000,7.100000,7.700000,48.000000,33.100000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.0,16.000000,52.000000,9.400000,10.100000,60.100000,44.100000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.0,23.000000,63.900000,10.000000,13.000000,72.000000,55.900000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.0,31.000000,77.500000,10.000000,39.300000,90.000000,71.600000,9.999000e+02,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [161]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = dnn.iloc[np.random.permutation(len(dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  temp  visib  wdsp   max   min   sndp  Apr  Aug  ...  May  Nov  \
233   2016   3  44.4   10.0  15.5  52.0  34.0  999.9    0    0  ...    0    0   
407   2013  23  36.3    9.7   7.0  41.0  30.9  999.9    0    0  ...    0    0   
1803  2016   9  59.9   10.0  16.7  68.0  54.0  999.9    0    0  ...    0    0   
2367  2015   6  73.5   10.0   9.3  81.0  66.9  999.9    0    1  ...    0    0   
2226  2018  13  69.1    7.7   6.2  75.9  64.9  999.9    0    1  ...    0    0   
1757  2016  14  62.2   10.0  12.9  72.0  54.0  999.9    0    0  ...    0    0   

      Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
233     0    0    0    1    0    0    0    0  
407     0    0    0    0    0    0    0    0  
1803    0    0    0    0    0    0    0    1  
2367    0    0    0    0    0    0    0    1  
2226    0    0    0    0    1    0    0    0  
1757    0    0    1    0    0    0    0    0  

[6 rows x 26 columns]


In [162]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

233     432
407     521
1803    686
2367    630
2226    617
1757    717
Name: NUM_COLLISIONS, dtype: int64


In [163]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)


26


In [164]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model', hidden_units=[17,13,9,7], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99c5e6510>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 41.121037, step = 1
INFO:tensorflow:global_step/sec: 350.821
INFO:tensorflow:loss = 0.815585, step = 101 (0.293 sec)
INFO:tensorflow:global_step/sec: 447.403
INFO:tensorflow:loss = 0.29131812, step = 201 (0.218 sec)
INFO:tensorflow:global_step/sec: 510.276
INFO:tensorflow:loss = 0.09008282, step = 301 (0.198 sec)
INFO:tensorflow:global_step/sec: 518.264
INFO:tensorflow:loss = 0.052201133, step = 401 (0.193 sec)
INFO:tensorflow:global_step/sec: 527.865
INFO:tensorflow:loss = 0.030631417, step = 501 (0.195 sec)
INFO:tensorflow:global_step/sec: 501.748
INFO:tensorflow:loss = 0.027305681, step = 601 (0.192 sec)
INFO:tensorflow:global_step/sec: 586.305
INFO:tensorflow:loss = 0.030179385, step = 701 (0.178 sec)
INFO:t

DNNRegression has RMSE of 78.24717634102431
Just using average = 599.5909980430529 has RMSE of 99.55323398451917


In [165]:
print(predictors[trainsize:].values)
#print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model', hidden_units=[17,13,9,7], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99bbfa950>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model', '_session_creation_timeout_secs': 7200}


[[2.018e+03 2.500e+01 4.530e+01 ... 0.000e+00 0.000e+00 1.000e+00]
 [2.013e+03 1.300e+01 2.810e+01 ... 1.000e+00 0.000e+00 0.000e+00]
 [2.014e+03 2.300e+01 5.950e+01 ... 0.000e+00 0.000e+00 1.000e+00]
 ...
 [2.018e+03 2.100e+01 4.850e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.014e+03 1.500e+01 6.490e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [2.016e+03 1.700e+01 4.150e+01 ... 0.000e+00 1.000e+00 0.000e+00]]


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5568788  0.5393471  0.5443441  0.43283385 0.41556218 0.573106
 0.5909341  0.5033253  0.5413491  0.5049177  0.47777614 0.54672474
 0.5639878  0.57162917 0.5412146  0.53128785 0.5016192  0.39523876
 0.46182284 0.5725149  0.5145943  0.531485   0.4341458  0.5636309
 0.56185985 0.58443606 0.4041516  0.49701694 0.5192967  0.4863644
 0.49219057 0.56869566 0.4372357  0.5065642  0.5068138  0.53947985
 0.513665   0.4480784  0.50305    0.46447203 0.43132603 0.52332467
 0.51088196 0.43565777 0.54483235 0.45655015 0.38729867 0.48060206
 0.5368682  0.5477901  0.5097679  0.5493637  0.54218733 0.42459548
 0.5168932  0.5528693  0.5290235  0.52981395 0.56537974 0.4876649
 0.5584972  0.43458077 0.4603415  0.58139634 0.52097136 0.52923566
 0.5807425  0.5560684  0.5416392  0.43494692 0.52106565 0.5496432
 0.5339975  0.50240594 0.48058462 0.48125815 0.52696425 0.5093319
 0.51559365 0.56482244 0.48939398 0.37166727 0.48819086 0.48747024
 0.54286915 0.55485094 0.55647826 0.5767821  0.55075634 0.49695644
 0

## Location


REF https://www.geeksforgeeks.org/read-a-zipped-file-as-a-pandas-dataframe/ 12/11

In [166]:
df_loc = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/locdnn.zip', index_col=0,compression='zip' )
print(df_loc[:6])

   year  da  NUM_COLLISIONS   latitude  longitude  temp  dewp     slp  visib  \
1  2018   2               1  40.681750 -73.967480  14.7   2.0  1024.9   10.0   
2  2018   2               1  40.645370 -73.945110  14.7   2.0  1024.9   10.0   
3  2018   2               1  40.614830 -73.998380  14.7   2.0  1024.9   10.0   
4  2018   2               1  40.592190 -74.087395  14.7   2.0  1024.9   10.0   
5  2018   2               1  40.769817 -73.782370  14.7   2.0  1024.9   10.0   
6  2018   2               1  40.660175 -73.928200  14.7   2.0  1024.9   10.0   

   wdsp  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  12.9  ...    0    0    0    0    1    0    0    0    0    0  
2  12.9  ...    0    0    0    0    1    0    0    0    0    0  
3  12.9  ...    0    0    0    0    1    0    0    0    0    0  
4  12.9  ...    0    0    0    0    1    0    0    0    0    0  
5  12.9  ...    0    0    0    0    1    0    0    0    0    0  
6  12.9  ...    0    0    0    0    1    0    0  

In [167]:
df_loc_dnn = df_loc.drop(columns=[ 'prcp', 'dewp','mxpsd','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["year"] != 2012]
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["year"] < 2020]
cols = df_loc_dnn['NUM_COLLISIONS']
df_loc_dnn = df_loc_dnn.drop(columns=['NUM_COLLISIONS'])
df_loc_dnn.insert(loc=28, column='NUM_COLLISIONS', value=cols)
print(df_loc_dnn[:6])
df_loc_dnn.describe()

   year  da   latitude  longitude  temp  visib  wdsp   max  min   sndp  ...  \
1  2018   2  40.681750 -73.967480  14.7   10.0  12.9  18.0  5.0  999.9  ...   
2  2018   2  40.645370 -73.945110  14.7   10.0  12.9  18.0  5.0  999.9  ...   
3  2018   2  40.614830 -73.998380  14.7   10.0  12.9  18.0  5.0  999.9  ...   
4  2018   2  40.592190 -74.087395  14.7   10.0  12.9  18.0  5.0  999.9  ...   
5  2018   2  40.769817 -73.782370  14.7   10.0  12.9  18.0  5.0  999.9  ...   
6  2018   2  40.660175 -73.928200  14.7   10.0  12.9  18.0  5.0  999.9  ...   

   Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
1    0    0    0    1    0    0    0    0    0               1  
2    0    0    0    1    0    0    0    0    0               1  
3    0    0    0    1    0    0    0    0    0               1  
4    0    0    0    1    0    0    0    0    0               1  
5    0    0    0    1    0    0    0    0    0               1  
6    0    0    0    1    0    0    0    0    0          

,year,da,latitude,longitude,temp,visib,wdsp,max,min,sndp,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,...,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06
mean,2.016149e+03,1.566747e+01,4.072402e+01,-7.392063e+01,5.203610e+01,8.262622e+00,1.066076e+01,6.012695e+01,4.442143e+01,9.999000e+02,...,8.510758e-02,8.877971e-02,8.587448e-02,1.487626e-01,1.173738e-01,1.438814e-01,1.597195e-01,1.467698e-01,1.505395e-01,1.026582e+00
std,1.970562e+00,8.752606e+00,7.845179e-02,8.651997e-02,1.410404e+01,2.220453e+00,4.197681e+00,1.424273e+01,1.468128e+01,2.842172e-12,...,2.790418e-01,2.844256e-01,2.801787e-01,3.558544e-01,3.218653e-01,3.509695e-01,3.663458e-01,3.538765e-01,3.575996e-01,1.784039e-01
min,2.013000e+03,1.000000e+00,4.049895e+01,-7.425453e+01,5.800000e+00,2.000000e-01,2.700000e+00,1.800000e+01,-2.000000e+00,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.014000e+03,8.000000e+00,4.066895e+01,-7.397644e+01,4.100000e+01,7.000000e+00,7.700000e+00,4.890000e+01,3.310000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,2.016000e+03,1.600000e+01,4.072234e+01,-7.392891e+01,5.300000e+01,9.300000e+00,1.010000e+01,6.100000e+01,4.500000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,2.018000e+03,2.300000e+01,4.076838e+01,-7.386641e+01,6.430000e+01,1.000000e+01,1.290000e+01,7.300000e+01,5.700000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,2.019000e+03,3.100000e+01,4.091288e+01,-7.366301e+01,7.750000e+01,1.000000e+01,3.930000e+01,9.000000e+01,7.160000e+01,9.999000e+02,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01


In [168]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_loc_dnn.iloc[np.random.permutation(len(df_loc_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

         year  da   latitude  longitude  temp  visib  wdsp   max   min   sndp  \
578269   2013  25  40.856180 -73.928714  55.3   10.0   4.5  64.0  44.1  999.9   
436968   2017  28  40.748350 -73.967865  69.2   10.0   8.5  77.0  55.9  999.9   
1257363  2016  16  40.749725 -73.983622  45.7    7.3  15.7  51.1  39.9  999.9   
854729   2016  11  40.773440 -73.920970  54.9   10.0  16.0  63.0  46.9  999.9   
1493674  2019  19  40.770510 -73.925470  62.3    6.7   6.8  69.1  57.9  999.9   
799817   2017  15  40.826855 -73.922676  43.9   10.0  11.3  48.9  39.0  999.9   

         ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
578269   ...    0    0    0    1    0    0    0    0    1    0  
436968   ...    0    0    0    0    0    0    0    1    0    0  
1257363  ...    0    0    0    0    0    0    0    0    0    0  
854729   ...    0    1    0    0    0    0    0    1    0    0  
1493674  ...    0    0    0    0    0    0    0    0    1    0  
799817   ...    0    1    0    0    0    0

In [169]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

578269     1
436968     1
1257363    1
854729     1
1493674    1
799817     1
Name: NUM_COLLISIONS, dtype: int64


In [170]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)


28


In [171]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_loc', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_loc', hidden_units=[16,7,3], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99ff2c410>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_loc', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_loc/model.ckpt.
INFO:tensorflow:loss = 8.462106e-07, step = 1
INFO:tensorflow:global_step/sec: 416.448
INFO:tensorflow:loss = 1.2511026e-08, step = 101 (0.244 sec)
INFO:tensorflow:global_step/sec: 487.272
INFO:tensorflow:loss = 1.715747e-08, step = 201 (0.207 sec)
INFO:tensorflow:global_step/sec: 521.822
INFO:tensorflow:loss = 3.1736526e-08, step = 301 (0.192 sec)
INFO:tensorflow:global_step/sec: 554.828
INFO:tensorflow:loss = 2.2673124e-08, step = 401 (0.181 sec)
INFO:tensorflow:global_step/sec: 519.208
INFO:tensorflow:loss = 9.552128e-09, step = 501 (0.188 sec)
INFO:tensorflow:global_step/sec: 574.79
INFO:tensorflow:loss = 2.2665516e-08, step = 601 (0.174 sec)
INFO:tensorflow:global_step/sec: 551.717
INFO:tensorflow:loss = 1.7947363e-08, step =

DNNRegression has RMSE of 0.38386723782836285
Just using average = 1.0266642526347887 has RMSE of 0.1763082830732086


In [172]:
print(predictors[trainsize:].values)
#print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_loc', hidden_units=[16,7,3], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99ca2ff90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_loc', '_session_creation_timeout_secs': 7200}


[[2.01900000e+03 6.00000000e+00 4.08398500e+01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [2.01300000e+03 1.60000000e+01 4.07432641e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01300000e+03 2.30000000e+01 4.07539280e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.01900000e+03 4.00000000e+00 4.06128500e+01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.01400000e+03 7.00000000e+00 4.08268488e+01 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01600000e+03 2.10000000e+01 4.07390820e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_loc/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.00117764 0.00117764 0.00117764 ... 0.00117764 0.00117764 0.00117764]
[0.00086133 0.00086133 0.00086133 ... 0.00086133 0.00086133 0.00086133]
The trained model has an aproximate error rate of -0.34098329754667495 which equates to -33%
